In [1]:
# installing requirements from txt file
#pip install -r requirements.txt

In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf
from datetime import datetime

# **Step 1: Data retrieval and indicators calculation**

The idea is to consider a portfolio made only of SPY ETF, as it has been seen that holding an ETF which replicates the Standard and Poor 500 can be one of the best investments you can make.

In [3]:
# downloading monthly prices of the SPY ETF, as VIX data will be monthly and therefore we keep returns as monthly
spy_prices = yf.download('SPY', start = '2000-01-01', end = '2024-12-31', interval = '1mo')
spy_prices = spy_prices['Adj Close']
spy_prices

[*********************100%%**********************]  1 of 1 completed


Date
2000-01-01     88.948547
2000-02-01     87.594208
2000-03-01     95.839798
2000-04-01     92.708809
2000-05-01     91.251266
                 ...    
2024-08-01    560.071289
2024-09-01    570.086792
2024-10-01    566.732605
2024-11-01    600.528809
2024-12-01    584.114075
Name: Adj Close, Length: 300, dtype: float64

In [4]:
spy_rets = spy_prices.pct_change().dropna()
spy_rets

Date
2000-02-01   -0.015226
2000-03-01    0.094134
2000-04-01   -0.032669
2000-05-01   -0.015722
2000-06-01    0.017286
                ...   
2024-08-01    0.023366
2024-09-01    0.017883
2024-10-01   -0.005884
2024-11-01    0.059633
2024-12-01   -0.027334
Name: Adj Close, Length: 299, dtype: float64

Next, we upload VIX future (UX1 index) term structure data downloaded from Bloomberg as of 1/17/2025:

In [9]:
vix_data = pd.read_excel('VIX_term_structure_20250117.xlsx', header = 0)
vix_data = vix_data.drop(vix_data.index[0]) # removing first unnecessary row
vix_data

,Tenor,Ticker,Period,Last Price,Days to expiration
1,Spot,VIX Index,Spot,15.9700,0.0
2,1M,UXF5 Index,01/2025,16.1792,30.0
3,1M,UXG5 Index,02/2025,17.2382,60.0
4,2M,UXH5 Index,03/2025,17.8351,90.0
5,3M,UXJ5 Index,04/2025,18.1962,120.0
6,4M,UXK5 Index,05/2025,18.4005,150.0
7,5M,UXM5 Index,06/2025,18.5484,180.0
8,6M,UXN5 Index,07/2025,18.8250,210.0
9,7M,UXQ5 Index,08/2025,18.8000,240.0
10,8M,UXU5 Index,09/2025,19.1000,270.0


In [50]:
def constant_maturity_term_structure(data, maturity_target): # TO MODIFY AND TO DO FOR ALL MATURITIES
    """
    This function computes the linear interpolation of VIX futures prices 
    for generating a constant maturity term structure.
    
    It takes the VIX dataframe with prices and days to expiration,
    together with a target maturity, expressed in days, as inputs,
    and will return the interpolated prices of the VIX futures.
    """
    before = data[data["Days to expiration"] <= maturity_target] # looking for the nearest earlier contract for interpolation
    after = data[data["Days to expiration"] > maturity_target] # looking for the nearest later contract for interpolation

    if before.empty or after.empty:
        raise ValueError("Missing near contracts needed for linear interpolation formula.")

    lower = before.iloc[-1] # selecting the earlier nearest contracts
    upper = after.iloc[0] # selecting the later nearest contracts

    # linear interpolation
    weight_upper = (maturity_target - lower["Days to expiration"]) / (upper["Days to expiration"] - lower["Days to expiration"])
    weight_lower = 1 - weight_upper # the weight of the earlier contract is the complementary of the other one

    return lower["Last Price"] * weight_lower + upper["Last Price"] * weight_upper # new interpolated price

In [51]:
constant_maturity_term_structure(vix_data, 30)

16.1792

In [ ]:
# creating functions for the three indicators which will compose the innovative part of our approach

def rolling_std(series, time_interval): # defining a function for volatility, which we consider as rolling standard deviation
    return series.rolling(window = time_interval).std()

def rolling_correlation(series1, series2, time_interval): # defining a function for the rolling correlation
    return series1.rolling(window = time_interval).corr(series2)

def ROC(series): # defining a function for the Rate Of Change
    return series.pct_change()